In [1]:
import sys
import json
import ctypes  # An included library with Python install.
from ipyleaflet import Marker, LayersControl, basemaps, GeoData, Icon, WidgetControl, Choropleth, Map, CircleMarker

from ipywidgets import HTML, IntSlider, Image, Dropdown, Label, Button, interact, jsdlink, Layout, Output, HBox, RadioButtons
from ipywidgets.embed import embed_data
import geopandas as gpd
import fiona
from shapely.geometry import shape
import glob
import folium
import ipywidgets as widgets
from branca.colormap import linear
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plot
import pandas as pd
from datetime import datetime
import time
import matplotlib
import numpy as np
import math
import plotly.graph_objects as go



In [2]:
if __name__ == '__main__':
    l2 = Label(value = "Spill Flow and Rainfall Intensity", layout=widgets.Layout(width='100%', align_items='stretch'))
    display(l2)
    rainfall_dataset = pd.read_csv("Data/Rainfall2002.csv", header=1)
    spillflow_dataset = pd.read_csv("Data/3 years - Spill Flow_adjusted.csv", header=1)
    start_date =  datetime.strptime(rainfall_dataset['Date/Time'].values[0], '%d/%m/%Y %H:%M') 
    end_date = datetime.strptime(rainfall_dataset['Date/Time'].values[len(rainfall_dataset)-1],'%d/%m/%Y %H:%M')
   
    dates = pd.date_range(start_date, end_date, freq='D')
    options = [date.strftime('%d %b %y') for date in dates]
    index = (0, len(options)-1)

    s1 = widgets.SelectionRangeSlider(
            options=options,
            index=index,
            description='Dates',
            orientation='horizontal',
            layout={'width': '500px'})
    graph_window = widgets.Output(layout = widgets.Layout(height='200px', width='600px'))
    def print_date_range(date_range):
        rainfall_dataset = pd.read_csv("Data/Rainfall2002.csv", header=1, parse_dates=['Date/Time'])
        spillflow_dataset = pd.read_csv("Data/3 years - Spill Flow_adjusted.csv", header=1, parse_dates=['Date/Time'])
        fig, ax1 = plot.subplots(1,1, figsize=(20,5)) 
        ax1.set_xlabel('Date', color = 'black', fontsize = 30 ) 
        ax1.set_ylabel('Spill Flow', color = 'black', fontsize = 30 ) 
        ax1 = sns.lineplot(x = "Date/Time", y = 'SSC_BrasmillLane Spill Flow m3/s', color = "black", data=spillflow_dataset)
        ax1.tick_params(axis='x', labelsize=20)
        ax1.tick_params(axis='y', labelsize=20)
        ax2 = ax1.twinx() 
        ax2 = sns.lineplot(x = "Date/Time", y = 'AvRainIntensity', color = "red", data=rainfall_dataset)
        ax2.set_ylabel('Rainfall Intensity', color = 'red', fontsize = 30) 
        ax2.tick_params(axis='y', labelsize=20, labelcolor = 'red')
        start_date = datetime.strptime(date_range[0],'%d %b %y')
        end_date = datetime.strptime(date_range[1],'%d %b %y')
        plot.xlim([start_date, end_date])
        with graph_window :
            graph_window.clear_output()
            plot.show()
        #print(date_range)

    widgets.interact(
        print_date_range,
        date_range=s1
    )
    display(graph_window)
    
    water_body_scenarios =  Dropdown(options=['3', '5', '7'], value='3', description='Water Body Type:', disabled=False)
    assessment_options = Dropdown(options=['No standard', '99th% standards', 'FIS & 99th% assessment'], value = 'No standard', description='Assess:')
    def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            if assessment_options.value == 'FIS & 99th% assessment':
                d_scenarios2.disabled = False
            else:
                d_scenarios2.disabled = True
                

    assessment_options.observe(on_change)

    d_scenarios1 = Dropdown(options=['Scenario 1', 'Scenario 2', 'Scenario 3', 'Scenario 4', 'Scenario 5', 'Scenario 6'], value='Scenario 1', description='Scen.:', disabled=False)
    d_scenarios2 = Dropdown(options=['Scenario 1', 'Scenario 2', 'Scenario 3', 'Scenario 4', 'Scenario 5', 'Scenario 6'], value='Scenario 1', description='Scen.:', disabled=True)
    d1 = Dropdown(options=['River Flow m3/s', 'River BODdis mg/l', 'River AmmN mg/l', 'River DO mg/l', 'River Temperature Deg C', 'DO %sat at downstream end of reach %'], value='River DO mg/l', description='Var.:', disabled=False)
    d2 = Dropdown(options=['Mean', 'Min', '5%ile', '10%ile', '20%ile', '50%ile',  '90%ile', '95%ile', '99%ile', 'Max', 'MeanDW', '5%ileDW', '10%ileDW', '90%ileDW', '95%ileDW'], value='10%ile', description='Metric:', disabled=False)
    b1 = Button(description='Plot Vis.', disabled=False, button_style='')
    bx1 = widgets.Checkbox(False, description='99th% standards')
    bx2 = widgets.Checkbox(False, description='FIS & 99th% assessment')
    b2 = Button(description='WQ impact between scenarios.', disabled=True, button_style='')
    #hbox_vis_options = HBox([b1, bx1])
    #hbox_scenarios = HBox([d_scenarios1, d_scenarios2, bx2])
    #hbox_scenario_comparison = HBox([d1, b2])
    display(water_body_scenarios)
    display(assessment_options)
    display(d_scenarios1)
    display(d_scenarios2)
    display(d1)
    display(d2)
    display(b1)
    colourbar_window = widgets.Output(layout = widgets.Layout(height='180px', width='1000px'))
    
    def Mbox(title, text, style):
        return ctypes.windll.user32.MessageBoxW(0, text, title, style)
    
    
    #define map
    m = Map(center=(52.3,8.0), zoom = 14, basemap = basemaps.Stamen.Toner, layout=Layout(width='100%', height='600px'))
         
    display(m)
    
    

    #add SSCs
    style={"color": "black",  "weight":1, "fillColor": "white"}
    hover_style={"color": "red",  "weight":2, "fillColor": "red"}
    
    html_SSC = HTML("Hover Over SSCs or River Reaches")
    html_SSC.layout.margin = "0px 20px 20px 20px"
    control = WidgetControl(widget=html_SSC, position="topright")
    m.add_control(control)
    
    def update_html_SSC_name(feature, **kwargs):
         html_SSC.value = """<h4>SSC: {}</h4>
          """.format(feature["properties"]["NAME"])
    
    # add river catchment
    river_catchment_style={"color": "green",  "weight":1, "fillColor": "white"}
    river_catchment_file = "GIS/RiverCatchment.shp"
    river_catchment = gpd.read_file(river_catchment_file)
    river_catchment_geo_shape = river_catchment.to_crs(4326)
    river_catchment_shape_to_map = GeoData(geo_dataframe = river_catchment_geo_shape, style = river_catchment_style)
    m.add_layer(river_catchment_shape_to_map) 
    #lonCent = (river_catchment_geo_shape.bounds.maxx + river_catchment_geo_shape.bounds.minx).mean()/2
    #latCent = (river_catchment_geo_shape.bounds.maxy + river_catchment_geo_shape.bounds.miny).mean()/2
    #m.center = (latCent,lonCent)
    
    #add SSC connectivity
    #SSC_conn_file = "GIS/SSC_links2.shp"
    #SSC_con = gpd.read_file(SSC_conn_file)
    #geo_SSC_con_shape = SSC_con.to_crs(4326)
    #geo_SSC_con_shape_to_map = GeoData(geo_dataframe = geo_SSC_con_shape, style = {"color": "green", "fillColor": "green", "weight":4})
    #m.add_layer(geo_SSC_con_shape_to_map)
    
    #add river subcatchments
    subcatchments_style={"color": "black",  "weight":1, "fillColor": "white"}
    subcatchments_file = "GIS/sub-catchments.shp"
    subcatchments = gpd.read_file(subcatchments_file)
    subcatchments_geo_shape = subcatchments.to_crs(4326)
    subcatchments_shape_to_map = GeoData(geo_dataframe = subcatchments_geo_shape, style = subcatchments_style)
    m.add_layer(subcatchments_shape_to_map)
    
    
    #add SCCs visualising spills
    spills = pd.read_csv("results/spills_for_visualisation.csv")
    spills_table = spills.groupby(spills.columns[0],as_index=False).size()
    spills_table.rename(columns={'size': 'Spills frequency'}, inplace=True)
    spills_table['SSC'] = spills_table['SSC'].str.replace('SSC_BartonBridge', 'SSC_BartonBridge_SC')
    spills_table['SSC'] = spills_table['SSC'].str.replace('SSC_ChurchSt', 'SSC_Church_St_SC')
    spills_table['SSC'] = spills_table['SSC'].str.replace('SSC_CulverSt', 'SSC_CulverSt_SC')
    spills_table['SSC'] = spills_table['SSC'].str.replace('SSC_SilverSt', 'SSC_SilverSt_SC')
    spills_table['SSC'] = spills_table['SSC'].str.replace('SSC_WooleySt', 'SSC_WooleySt_SC')
    spills_dic = {}
    spills = []
    for i in range(0, len(spills_table)):
        spills_dic[spills_table["SSC"].values[i]] = spills_table["Spills frequency"].values[i]
        spills.append(spills_table["Spills frequency"].values[i])
    spills_dic["SSC_BartonFarm_SC"] = 0
    spills.append(0)
    geojson_data = json.load(open("GIS/SSCs/collated_SSCs.json","r"))
    for feature in geojson_data["features"]:
        properties = feature["properties"]
        feature.update(id=properties["NAME"])
    hover_style={"color": "red",  "weight":2, "fillColor": "red"}
    layer = Choropleth(geo_data=geojson_data, choro_data=spills_dic, colormap=linear.Blues_05, style={'fillOpacity': 0.5, "color":"black"}, hover_style = hover_style)
    layer.on_hover(update_html_SSC_name)
    m.add_layer(layer)
    
    ssc_shape = gpd.read_file("GIS/SSCs/collated_SSCs.shp")
    geo_shape = ssc_shape.to_crs(4326)
    lonCent = (geo_shape.bounds.maxx + geo_shape.bounds.minx).mean()/2
    latCent = (geo_shape.bounds.maxy + geo_shape.bounds.miny).mean()/2
    m.center = (latCent,lonCent)
    
    #add river shape file to map
    def update_html_river_reach_name(feature, **kwargs):
         html_SSC.value = """<h4>River Reach: {}</h4>
            """.format(feature["properties"]["name1"])
                                                     
    river_file = "GIS/RiverReaches.shp"
    river_shape = gpd.read_file(river_file)
    river_geo_shape = river_shape.to_crs(4326)
    river_style={"color": "blue",  "weight":2, "fillColor": "blue"}
    river_hover_style={"color": "red",  "weight":4, "fillColor": "red"}
    river_shape_to_map = GeoData(geo_dataframe = river_geo_shape, style = river_style, hover_style = river_hover_style)
    river_shape_to_map.on_hover(update_html_river_reach_name)
    m.add_layer(river_shape_to_map)
    
    #add SCC links
    #links_file = "GIS/links.shp"
    #links_shape = gpd.read_file(links_file)
    #links_geo_shape = links_shape.to_crs(4326)
    #links_style={"color": "brown",  "weight":2, "fillColor": "white"}
    #links_shape_to_map = GeoData(geo_dataframe = links_geo_shape, style = links_style)
    #m.add_layer(links_shape_to_map)
     
    #add STWs
    STW_point_file = "GIS/STW_locations_4326.shp"
    icon_url = "https://icons.iconarchive.com/icons/custom-icon-design/pretty-office-9/128/triangle-icon.png"
    STW_point_icon = Icon(icon_url=icon_url, icon_size=[20, 20])
    with fiona.open(STW_point_file) as src:
        for feature in src:
            #print(feature)
            properties = feature['properties']
            geom = shape(feature['geometry'])
            STW_name = properties["SimName"]
            yx_coords = [geom.y, geom.x]
            mark = Marker(location=yx_coords, icon = STW_point_icon, title = STW_name)
            message = HTML(value="%s"%(STW_name))
            mark.popup = message
            m.add_layer(mark)
    #add CSOs        
    CSO_point_file = "GIS/CSOs_4326.shp"
    CSO_icon_url = "https://icons.iconarchive.com/icons/fatcow/farm-fresh/32/shape-square-icon.png"
    CSO_point_icon = Icon(icon_url=CSO_icon_url, icon_size=[20, 20], color = "green")
    with fiona.open(CSO_point_file) as src:
        for feature in src:
            #print(feature)
            properties = feature['properties']
            geom = shape(feature['geometry'])
            ID = properties["node_id"]
            yx_coords = [geom.y, geom.x]
            mark = Marker(location=yx_coords, icon = CSO_point_icon, title = ID)
            message = HTML(value="%s"%(ID))
            mark.popup = message
            m.add_layer(mark)
            
    #add nodes        
    nodes_point_file = "GIS/nodes_4326.shp"
    nodes_icon_url = "https://icons.iconarchive.com/icons/sekkyumu/developpers/32/Orange-Ball-icon.png"
    nodes_point_icon = Icon(icon_url=nodes_icon_url, icon_size=[5, 5])
    with fiona.open(nodes_point_file) as src:
        for feature in src:
            #print(feature)
            properties = feature['properties']
            geom = shape(feature['geometry'])
            yx_coords = [geom.y, geom.x]
            mark = Marker(location=yx_coords, icon = nodes_point_icon)
            #m.add_layer(mark)
            
    colourbar_window = widgets.Output(layout = widgets.Layout(height='300px', width='1000px'))
    
    river_quality = pd.read_csv("results/river_water_quality.csv")
    filtered_river_quality = river_quality[river_quality["Var"] == d1.value]
    filtered_river_quality = filtered_river_quality[filtered_river_quality["Metric"] == d2.value]
    #print(filtered_river_quality.head())
    reaches = filtered_river_quality["Reach"].to_list()
    values = filtered_river_quality["Value"].to_list()
    for i in range(0, len(reaches)):
        reaches[i] = reaches[i].replace("RR", "")
        pos = reaches[i].index("_",0)
        reaches[i] = reaches[i][: pos] + str(math.trunc(int(reaches[i][pos + 1:len(reaches[i])])/10))
    mapping_data = dict(zip(reaches, values))
    #add river reach points
    river_point_file = "GIS/River_points.shp"
    icon_url = "https://icons.iconarchive.com/icons/custom-icon-design/pretty-office-9/128/circle-icon.png"
    river_point_icon = Icon(icon_url=icon_url, icon_size=[20, 20])
    circle_marker_list = []
    reach_names = []
    with fiona.open(river_point_file) as src:
        for feature in src:
            #print(feature)
            properties = feature['properties']
            geom = shape(feature['geometry'])
            reach_name = properties["NAME"]
            yx_coords = [geom.y, geom.x]
            mark = Marker(location=yx_coords, icon = river_point_icon, title = reach_name)
            circle_marker = CircleMarker()
            circle_marker.location = (geom.y, geom.x)
            circle_marker.radius = 5
            circle_marker.color = "blue"
            circle_marker.fill_color = "blue"
            circle_marker.fill_opacity = 1
            m.add_layer(circle_marker)
            circle_marker_list.append(circle_marker)
            reach_name = reach_name.replace("_","")
            reach_names.append(reach_name)
    cmap = matplotlib.cm.get_cmap('jet')
    norm = matplotlib.colors.Normalize(0, max(values))
    color_producer = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)
    j = 0
    for circle in circle_marker_list:
        index = mapping_data[reach_names[j]]
        rgba = color_producer.to_rgba(index)
        circle.color = matplotlib.colors.to_hex(rgba)
        circle.fill_color = matplotlib.colors.to_hex(rgba)
        circle.radius = math.trunc(index/max(values) * 10) 
        circle.name = str(index)
        j +=1
    #fig = plot.figure()
    #ax = fig.add_axes([0.05, 0.80, 0.9, 0.1])
    #col_map = plot.get_cmap('jet')

    
           
    
    fig = plot.figure()
    ax = fig.add_axes([0.05, 0.80, 0.9, 0.1])
    col_map = plot.get_cmap('Blues')
    cb = matplotlib.colorbar.ColorbarBase(ax, orientation='horizontal', 
                               cmap=col_map,
                               norm=matplotlib.colors.Normalize(0, max(spills)),  # vmax and vmin
                               ticks=[0, max(spills)/2 , max(spills)])
    cb.set_label('Average Spill Frequency (spills/yr)')
    
    fig2 = plot.figure()
    ax2 = fig2.add_axes([0.05, 0.80, 0.9, 0.1])
    col_map2 = plot.get_cmap('jet')


    cb2 = matplotlib.colorbar.ColorbarBase(ax2, orientation='horizontal', 
                               cmap=col_map2,
                               norm=matplotlib.colors.Normalize(0, max(values)),  # vmax and vmin
                               ticks=[0, max(values)/2 , max(values)])
    cb2.set_label(d1.value + ": " + d2.value)
    #print(values) 
    #display(colourbar_window)
    with colourbar_window :
        colourbar_window.clear_output()
        plot.show() 
    display(colourbar_window)
    
    percentile_standards_BOD = []
    percentile_standards_AMM = []
    percentile_standards_BOD.append({"Status": "Poor", "Colour": "brown", "min": 14, "max": 99999999})
    percentile_standards_BOD.append({"Status": "Moderate", "Colour": "#ffbf00", "min": 11, "max": 14})
    percentile_standards_BOD.append({"Status": "Good", "Colour": "#87a96b", "min": 9, "max": 11})
    percentile_standards_BOD.append({"Status": "High", "Colour": "#cfcfc4", "min": 0, "max": 9})
    percentile_standards_BOD = pd.DataFrame(percentile_standards_BOD)
    percentile_standards_AMM.append({"Status": "Poor", "brown": "(153, 76, 0)", "min": 2.6, "max": 99999999})
    percentile_standards_AMM.append({"Status": "Moderate", "Colour": "#ffbf00", "min": 1.5, "max": 2.6})
    percentile_standards_AMM.append({"Status": "Good", "Colour": "#87a96b", "min": 0.7, "max": 1.5})
    percentile_standards_AMM.append({"Status": "High", "Colour": "#cfcfc4", "min": 0, "max": 0.7})
   
    percentile_standards_AMM = pd.DataFrame(percentile_standards_AMM)
    
    output = widgets.Output()
    def on_button_clicked(b):
        with output:
            flag = 0
            river_quality = pd.read_csv("results/river_water_quality.csv")
            filtered_river_quality = river_quality[river_quality["Var"] == d1.value]
            filtered_river_quality = filtered_river_quality[filtered_river_quality["Metric"] == d2.value]
            reaches = filtered_river_quality["Reach"].to_list()
            values = filtered_river_quality["Value"].to_list()
            for i in range(0, len(reaches)):
                reaches[i] = reaches[i].replace("RR", "")
                pos = reaches[i].index("_",0)
                reaches[i] = reaches[i][: pos] + str(math.trunc(int(reaches[i][pos + 1:len(reaches[i])])/10))
            if assessment_options.value in ('99th% standards', 'FIS & 99th% assessment'):
                if d1.value in ("River BODdis mg/l","River AmmN mg/l") and d2.value == "99%ile":
                    if assessment_options.value == '99th% standards':
                        #Mbox('Valid selection', 'Valid variable or metric selected', 1)
                        flag = 1
                        mapping_data = dict(zip(reaches, values))
                        j = 0
                        for circle in circle_marker_list:
                            index = mapping_data[reach_names[j]]
                            data = None
                            colour = "white"
                            if d1.value == "River BODdis mg/l":
                                data = percentile_standards_BOD
                            else: 
                                data = percentile_standards_AMM
                            for i in range(0, len(data)):
                                if index >= data["min"].values[i] and index <= data["max"].values[i]:
                                    colour = data["Colour"].values[i]
                            circle.color = colour
                            circle.fill_color = colour
                            circle.radius = math.trunc(index/max(values) * 10) 
                            circle.name = str(index)
                            j +=1
                else:
                    Mbox('Invalid selection', 'Invalid variable or metric selected', 1)
                    return 
            else:
            #print("Got here")
                mapping_data = dict(zip(reaches, values))
                cmap = matplotlib.cm.get_cmap('jet')
                norm = matplotlib.colors.Normalize(0, max(values))
                color_producer = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)
                j = 0
                for circle in circle_marker_list:
                    index = mapping_data[reach_names[j]]
                    rgba = color_producer.to_rgba(index)
                    circle.color = matplotlib.colors.to_hex(rgba)
                    #circle.color = "red"
                    circle.fill_color = matplotlib.colors.to_hex(rgba)
                    circle.radius = math.trunc(index/max(values) * 10) 
                    circle.name = str(index)
                    j +=1
                fig2 = plot.figure()
                ax2 = fig2.add_axes([0.05, 0.80, 0.9, 0.1])
                col_map2 = plot.get_cmap('jet')
                cb2 = matplotlib.colorbar.ColorbarBase(ax2, orientation='horizontal', 
                   cmap=col_map2,
                   norm=matplotlib.colors.Normalize(0, max(values)),  # vmax and vmin
                   ticks=[0, max(values)/2 , max(values)])
                cb2.set_label(d1.value + ": " + d2.value)
            
            fig = plot.figure()
            ax = fig.add_axes([0.05, 0.80, 0.9, 0.1])
            col_map = plot.get_cmap('Blues')
            cb = matplotlib.colorbar.ColorbarBase(ax, orientation='horizontal', 
               cmap=col_map,
               norm=matplotlib.colors.Normalize(0, max(spills)),  # vmax and vmin
               ticks=[0, max(spills)/2 , max(spills)])
            cb.set_label('Average Spill Frequency (spills/yr)')
    
            
            with colourbar_window:
                colourbar_window.clear_output()
                plot.show()
                if flag == 1:
                    file = open("figures/Table_6.99.jpg", "rb")
                    image = file.read()
                    i1 = Image(value=image, format='png', width=200, height=58)
                    display(i1)
             
    b1.on_click(on_button_clicked)
    m.save('my_map.html', title='My Map')
   

Label(value='Spill Flow and Rainfall Intensity', layout=Layout(align_items='stretch', width='100%'))

interactive(children=(SelectionRangeSlider(description='Dates', index=(0, 364), layout=Layout(width='500px'), …

Output(layout=Layout(height='200px', width='600px'))

Dropdown(description='Water Body Type:', options=('3', '5', '7'), value='3')

Dropdown(description='Assess:', options=('No standard', '99th% standards', 'FIS & 99th% assessment'), value='N…

...

Dropdown(description='Var.:', index=3, options=('River Flow m3/s', 'River BODdis mg/l', 'River AmmN mg/l', 'Ri…

Dropdown(description='Metric:', index=3, options=('Mean', 'Min', '5%ile', '10%ile', '20%ile', '50%ile', '90%il…

Button(description='Plot Vis.', style=ButtonStyle())

Map(center=[52.3, 8.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

Output(layout=Layout(height='300px', width='1000px'))